# Creation of E/Z dataset

## Import section

In [2]:
import os
import pickle

## Loading of dataset.

choice: rs dataset as most datasets with only one double bond with e/z label

In [4]:
mode = 'val'
with open(os.path.join("hyperoptimization/src/rs/raw/", f"{mode}.pickle"), 'rb') as f:
    df = pickle.load(f)

In [5]:
df

,ID,SMILES_nostereo,rdkit_mol_cistrans_stereo,RS_label,RS_label_binary
0,Br[C@@]1(c2cccc(-c3ccccc3)c2)C=CC=CC1,BrC1(c2cccc(-c3ccccc3)c2)C=CC=CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BF0C6...,R,0
1,Br[C@@]1(c2cccc(-c3ccccc3)c2)C=CC=CC1,BrC1(c2cccc(-c3ccccc3)c2)C=CC=CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BF0B8...,R,0
2,Br[C@@]1(c2cccc(-c3ccccc3)c2)C=CC=CC1,BrC1(c2cccc(-c3ccccc3)c2)C=CC=CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BF0B8...,R,0
3,Br[C@@]1(c2cccc(-c3ccccc3)c2)C=CC=CC1,BrC1(c2cccc(-c3ccccc3)c2)C=CC=CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BF0B8...,R,0
4,Br[C@@]1(c2cccc(-c3ccccc3)c2)C=CC=CC1,BrC1(c2cccc(-c3ccccc3)c2)C=CC=CC1,<rdkit.Chem.rdchem.Mol object at 0x000001BF0B8...,R,0
...,...,...,...,...,...
70094,c1csc(CNCC[C@H]2CCCNC2)n1,c1csc(CNCCC2CCCNC2)n1,<rdkit.Chem.rdchem.Mol object at 0x000001BF433...,R,0
70095,c1csc(CNCC[C@H]2CCCNC2)n1,c1csc(CNCCC2CCCNC2)n1,<rdkit.Chem.rdchem.Mol object at 0x000001BF433...,R,0
70096,c1csc(CNCC[C@H]2CCCNC2)n1,c1csc(CNCCC2CCCNC2)n1,<rdkit.Chem.rdchem.Mol object at 0x000001BF433...,R,0
70097,c1csc(CNCC[C@H]2CCCNC2)n1,c1csc(CNCCC2CCCNC2)n1,<rdkit.Chem.rdchem.Mol object at 0x000001BF433...,R,0


## Filtering only molecules with one double bond that is a stereocenter (and has 4 constituents)

In [ ]:
def filter_has_one_marked_stereo(df_entry, smiles_name: str = 'ID') -> bool:
    from ptgnn.features.chienn.molecule3d import smiles_to_3d_mol
    from rdkit import Chem

    # define allowed stereo types
    allowed_stereo = [
        Chem.rdchem.BondStereo.STEREOZ,
        Chem.rdchem.BondStereo.STEREOE
    ]

    # extract smiles string
    smiles = df_entry[smiles_name]

    # get molecule
    molecule = smiles_to_3d_mol(
        smiles=smiles,
        max_number_of_atoms=100,
        max_number_of_attempts=100
    )

    # determine whether molecule has exactly one double bond with marked E/Z
    from rdkit import Chem
    bonds = [
        (bond.GetBeginAtom().GetIdx(), bond.getEndAtom)
        for bond in molecule.GetBonds()
        if (bond.GetBondType() == Chem.rdchem.BondType.DOUBLE) and
           (bond.GetStereo() in allowed_stereo) and
           (bond.GetBeginAtom().GetDegree() == 3) and
           (bond.GetEndAtom().GetDegree() == 3) # reduces to 1/6th of items with previous ones
    ]